In [10]:
import os
import torch
from torch import nn, optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm

DATA_DIR = '/kaggle/input/fruit-dataset1/fruit_dataset/fruit_dataset'
BATCH_SIZE = 32
NUM_EPOCHS = 30
IMAGE_SIZE = 224
VAL_SPLIT = 0.15
MODEL_SAVE_DIR = './checkpoint'
LEARNING_RATE = 1e-4
WEIGHT_DECAY = 1e-5
EARLY_STOPPING_PATIENCE = 3
os.makedirs(MODEL_SAVE_DIR, exist_ok=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_transforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.RandomRotation(25),
    transforms.RandomHorizontalFlip(),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.ColorJitter(brightness=0.3, contrast=0.3),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

full_dataset = datasets.ImageFolder(DATA_DIR, transform=train_transforms)
num_classes = len(full_dataset.classes)
val_size = int(VAL_SPLIT * len(full_dataset))
train_size = len(full_dataset) - val_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])
val_dataset.dataset.transform = val_transforms
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

model = models.resnet18(pretrained=True)
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 256),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(256, num_classes)
)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5)

best_val_acc = 0.0
early_stopping_counter = 0

# === Training Loop ===
for epoch in range(NUM_EPOCHS):
    print(f"\nEpoch {epoch + 1}/{NUM_EPOCHS}")
    model.train()
    train_loss, train_correct = 0.0, 0

    for images, labels in tqdm(train_loader, desc="Training"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)
        train_correct += (outputs.argmax(1) == labels).sum().item()

    train_loss /= len(train_loader.dataset)
    train_acc = train_correct / len(train_loader.dataset)

    # Validation
    model.eval()
    val_loss, val_correct = 0.0, 0
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Validating"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * images.size(0)
            val_correct += (outputs.argmax(1) == labels).sum().item()

    val_loss /= len(val_loader.dataset)
    val_acc = val_correct / len(val_loader.dataset)
    scheduler.step(val_loss)

    print(f"Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f}")
    print(f"Val   Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")

    # Save every epoch
    torch.save(model.state_dict(), f"{MODEL_SAVE_DIR}/epoch_{epoch+1}.pt")

    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), f"{MODEL_SAVE_DIR}/best_model.pt")
        print(f"Best model updated at epoch {epoch+1}!")
        early_stopping_counter = 0
    else:
        early_stopping_counter += 1
        if early_stopping_counter >= EARLY_STOPPING_PATIENCE:
            print("Early stopping triggered.")
            break

print("\nTraining complete. Checkpoints saved in './checkpoint'.")


Epoch 1/30


Validating: 100%|██████████| 7/7 [00:07<00:00,  1.08s/it]


Train Loss: 1.7781, Acc: 0.4440
Val   Loss: 1.1484, Acc: 0.6825
Best model updated at epoch 1!

Epoch 2/30


Validating: 100%|██████████| 7/7 [00:07<00:00,  1.09s/it]


Train Loss: 0.8332, Acc: 0.8043
Val   Loss: 0.7181, Acc: 0.8294
Best model updated at epoch 2!

Epoch 3/30


...

Train Loss: 0.0878, Acc: 0.9858
Val   Loss: 0.5482, Acc: 0.8199

Epoch 6/30


Validating: 100%|██████████| 7/7 [00:07<00:00,  1.08s/it]


Train Loss: 0.0628, Acc: 0.9916
Val   Loss: 0.5048, Acc: 0.8436

Epoch 7/30


Validating: 100%|██████████| 7/7 [00:07<00:00,  1.09s/it]

Train Loss: 0.0371, Acc: 0.9967
Val   Loss: 0.4873, Acc: 0.8626
Early stopping triggered.

Training complete. Checkpoints saved in './checkpoint'.


In [4]:
import os
import torch
from torch import nn, optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm

DATA_DIR = '/kaggle/input/fruit-dataset1/fruit_dataset/fruit_dataset'
BATCH_SIZE = 32
NUM_EPOCHS = 50
IMAGE_SIZE = 224
VAL_SPLIT = 0.15
MODEL_SAVE_DIR = './checkpoint1'
LEARNING_RATE = 1e-3  # Higher initial learning rate
WEIGHT_DECAY = 1e-4
EARLY_STOPPING_PATIENCE = 8
os.makedirs(MODEL_SAVE_DIR, exist_ok=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Simplified data augmentation - too much augmentation was hurting performance
train_transforms = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.RandomRotation(15),  # Reduced rotation
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # ImageNet normalization
])

val_transforms = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


full_dataset = datasets.ImageFolder(DATA_DIR, transform=train_transforms)
num_classes = len(full_dataset.classes)
print(f"Number of classes: {num_classes}")
print(f"Class names: {full_dataset.classes}")

val_size = int(VAL_SPLIT * len(full_dataset))
train_size = len(full_dataset) - val_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])


val_dataset.dataset.transform = val_transforms

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

print(f"Train samples: {len(train_dataset)}, Val samples: {len(val_dataset)}")

class FruitClassifier(nn.Module):
    def __init__(self, num_classes):
        super(FruitClassifier, self).__init__()
        
        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            
            # Block 2
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            
            # Block 3
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            
            # Block 4
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
        )
        
        # Global Average Pooling instead of large linear layers
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        
        # Classification layers
        self.classifier = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )
        
        # Initialize weights
        self._initialize_weights()
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)
    
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

model = FruitClassifier(num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

# Learning rate scheduler
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1)

# Training tracking
best_val_acc = 0.0
early_stopping_counter = 0
train_losses = []
val_losses = []
train_accs = []
val_accs = []

print("Starting training...")
print(f"Device: {device}")

for epoch in range(NUM_EPOCHS):
    print(f"\nEpoch {epoch + 1}/{NUM_EPOCHS}")
    print(f"Learning Rate: {optimizer.param_groups[0]['lr']:.6f}")
    
    # Training phase
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    
    for batch_idx, (images, labels) in enumerate(tqdm(train_loader, desc="Training")):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()
        
        # Print progress every 10 batches
        if batch_idx % 10 == 0:
            print(f'Batch {batch_idx}, Loss: {loss.item():.4f}')
    
    # Calculate training metrics
    train_loss = train_loss / len(train_loader)
    train_acc = train_correct / train_total
    
    # Validation phase
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Validating"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
    
    val_loss = val_loss / len(val_loader)
    val_acc = val_correct / val_total
    
    # Update learning rate
    scheduler.step()
    
    # Store metrics
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs.append(train_acc)
    val_accs.append(val_acc)
    
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
    print(f"Overfitting gap: {train_acc - val_acc:.4f}")
    
    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'best_val_acc': best_val_acc,
            'train_losses': train_losses,
            'val_losses': val_losses,
            'train_accs': train_accs,
            'val_accs': val_accs
        }, f"{MODEL_SAVE_DIR}/best_model.pt")
        print(f"Best model updated! Val Acc: {val_acc:.4f}")
        early_stopping_counter = 0
    else:
        early_stopping_counter += 1
    
    # Early stopping
    if early_stopping_counter >= EARLY_STOPPING_PATIENCE:
        print("Early stopping triggered.")
        break
    
    # Save checkpoint every 5 epochs
    if (epoch + 1) % 5 == 0:
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'best_val_acc': best_val_acc,
        }, f"{MODEL_SAVE_DIR}/checkpoint_epoch_{epoch+1}.pt")

print(f"\nTraining complete!")
print(f"Best validation accuracy: {best_val_acc:.4f}")
print(f"Model saved to {MODEL_SAVE_DIR}/best_model.pt")

Number of classes: 10
Class names: ['Apple', 'Banana', 'Grapes', 'Guava', 'Mango', 'Orange', 'Papaya', 'Pomegranate', 'Strawberry', 'Watermelon']
Train samples: 1196, Val samples: 211
Starting training...
Device: cuda

Epoch 1/50
Learning Rate: 0.001000


Training:   3%|▎         | 1/38 [00:01<00:57,  1.55s/it]

Batch 0, Loss: 2.2994


Training:  29%|██▉       | 11/38 [00:08<00:20,  1.30it/s]

Batch 10, Loss: 2.0264


...

Batch 20, Loss: 1.7075


Training:  82%|████████▏ | 31/38 [00:18<00:04,  1.42it/s]

Batch 30, Loss: 1.1653


Validating: 100%|██████████| 7/7 [00:04<00:00,  1.50it/s]

Train Loss: 1.3419, Train Acc: 0.4749
Val Loss: 1.2753, Val Acc: 0.4645
Overfitting gap: 0.0105
Early stopping triggered.

Training complete!
Best validation accuracy: 0.4882
Model saved to ./checkpoint1/best_model.pt


In [6]:
import torch
from torchvision import transforms
from PIL import Image
import os
import sys

MODEL_PATH = './checkpoint1/best_model.pt'
IMAGE_PATH = '/kaggle/input/fruit-dataset1/mango.jpg'
IMAGE_SIZE = 224
CLASS_NAMES = ['Apple', 'Banana', 'Grapes', 'Guava', 'Mango', 'Orange', 'Papaya', 'Pomegranate', 'Strawberry', 'Watermelon']

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === Preprocessing Transform (same as validation transform) ===
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], 
                         [0.229, 0.224, 0.225])
])

import torch.nn as nn

class FruitClassifier(nn.Module):
    def __init__(self, num_classes):
        super(FruitClassifier, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# === Load Model ===
model = FruitClassifier(num_classes=len(CLASS_NAMES))
checkpoint = torch.load(MODEL_PATH, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model.to(device)
model.eval()

if not os.path.exists(IMAGE_PATH):
    print(f"Error: Image file '{IMAGE_PATH}' does not exist.")
    sys.exit(1)

image = Image.open(IMAGE_PATH).convert('RGB')
image = transform(image).unsqueeze(0).to(device)  # Add batch dimension

with torch.no_grad():
    outputs = model(image)
    _, predicted = torch.max(outputs, 1)
    predicted_class = CLASS_NAMES[predicted.item()]

print(f"Predicted Fruit: {predicted_class}")

Predicted Fruit: Mango


In [1]:
import torch
from torchvision import transforms, models
from PIL import Image
import os

MODEL_PATH = './checkpoint/best_model.pt'
IMAGE_PATH = '/kaggle/input/fruit-dataset1/apple.jpeg'  
IMAGE_SIZE = 224
CLASS_NAMES = ['Apple', 'Banana', 'Grapes', 'Guava', 'Mango', 'Orange', 'Peach', 'Pomegranate', 'Strawberry', 'Watermelon']  

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# === Load Model ===
model = models.resnet18(pretrained=False)
model.fc = torch.nn.Sequential(
    torch.nn.Linear(model.fc.in_features, 256),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.3),
    torch.nn.Linear(256, len(CLASS_NAMES))
)
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.to(device)
model.eval()

# === Load and Preprocess Image ===
image = Image.open(IMAGE_PATH).convert('RGB')
input_tensor = transform(image).unsqueeze(0).to(device)

# === Inference ===
with torch.no_grad():
    output = model(input_tensor)
    predicted_idx = output.argmax(1).item()
    predicted_class = CLASS_NAMES[predicted_idx]

print(f"Predicted Fruit: {predicted_class}")

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


FileNotFoundError: [Errno 2] No such file or directory: './checkpoint/best_model.pt'

In [7]:
import shutil
import os

# Create a zip archive of the checkpoint folder
shutil.make_archive('/kaggle/working/checkpoint_backup1', 'zip', '/kaggle/working/checkpoint1')

# The zip file will be saved as checkpoint_backup.zip in /kaggle/working/
# You can then download it from the Kaggle interface

'/kaggle/working/checkpoint_backup1.zip'